<a href="https://colab.research.google.com/github/vedantzope9/ML_mspa/blob/main/MSPA_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
#For Labelled dataset
df = pd.read_csv("diabetes.csv", error_bad_lines=False)
df.head()


<ipython-input-9-3b60a33b5da5>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("diabetes.csv", error_bad_lines=False)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
# Linear Regression
df=df.dropna()
x = df.drop('Outcome', axis=1)
y = df['Outcome']

In [11]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [12]:

model = tf.keras.Sequential([
   tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(1)
])

In [13]:
model.compile(optimizer='sgd', loss='mean_squared_error')


In [14]:
model.fit(X_train, y_train, epochs=100, verbose=0)

In [15]:
y_pred = model.predict(X_test)

5/5 [==============================] - 0s 4ms/step


In [16]:
mse=mean_squared_error(y_test,y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.16899596150539412


In [17]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [18]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [19]:
y_pred_knn = knn.predict(X_test)


In [20]:
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("k-NN Accuracy:", accuracy_knn)

k-NN Accuracy: 0.6948051948051948


In [22]:
#For Uabelled dataset
netflix_data=pd.read_csv("netflix_titles.csv")
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [23]:
#KMeans
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Preprocess descriptions, fill missing values with empty strings
corpus = netflix_data['description'].fillna('')

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# Convert the TF-IDF matrix to a dense NumPy array
tfidf_dense = tfidf_matrix.toarray()

# Define the number of clusters (e.g., 5 clusters)
n_clusters = 5

# Initialize cluster centroids by randomly selecting data points
initial_indices = np.random.choice(tfidf_dense.shape[0], n_clusters, replace=False)
initial_centroids = tf.Variable(tf.constant(tfidf_dense[initial_indices], dtype=tf.float32))

# K-Means clustering loop
for _ in range(100):
    distances = tf.reduce_sum(tf.square(tf.expand_dims(tf.constant(tfidf_dense, dtype=tf.float32), axis=1)) - tf.expand_dims(initial_centroids, axis=0), axis=2)

    # Assign data points to the nearest cluster
    cluster_assignments = tf.argmin(distances, axis=1)

    # Update cluster centroids
    new_centroids = tf.stack([
      tf.reduce_mean(tf.boolean_mask(tf.constant(tfidf_dense, dtype=tf.float32), tf.equal(cluster_assignments, i), axis=0))
      for i in range(n_clusters)
    ], axis=0)

    # Assign the updated centroids
    # Assign the updated centroids
initial_centroids.assign(tf.reshape(new_centroids, initial_centroids.shape))


# Get cluster assignments for the descriptions
cluster_labels = cluster_assignments.numpy()

# Add cluster labels to the DataFrame
netflix_data['cluster'] = cluster_labels

# Check the results
print(netflix_data[['title', 'description', 'cluster']])
